In [8]:
import pandas as pd 
import os
from sklearn.model_selection import KFold

### create task of FFPE 71 TCGA BRCA slides for train and val 5 fold split. Then test on HGSOC seperated primary and metastatic 

In [9]:
ffpe_BRCA = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/BRCA_pos_neg_FFPE.csv")
ffpe_BRCA.head()

,case_id,slide_id,slide_type,brca_status,hrd_score,Signature.1,Signature.2,Signature.3,Signature.5,Signature.8,...,SBS26,SBS35,SBS38,SBS39,SBS40,SBS41,ID1,ID2,ID4,ID8
0,TCGA-57-1994,TCGA-57-1994-01Z-00-DX1.A0798185-30C5-4C32-B1A...,DX1,negative,0.418119,-0.102017,-0.371777,-1.205535,-0.008223,1.038977,...,-0.453437,-0.394365,-0.076842,1.160019,-1.207158,0.786210,-0.472679,-0.355317,-0.066808,-0.924600
1,TCGA-23-1027,TCGA-23-1027-01Z-00-DX1.53F9DFF4-6811-4184-B2F...,DX1,positive,0.960062,0.287663,-0.371777,0.021880,-0.234356,0.495860,...,-0.453437,2.477546,-0.076842,-0.775756,-1.207158,0.432319,-0.472679,-0.355317,-0.066808,-0.068554
2,TCGA-42-2593,TCGA-42-2593-01Z-00-DX1.9d506f1d-a5b3-4643-bd1...,DX1,negative,0.708361,1.290638,-0.371777,-0.140401,-2.087248,-1.015064,...,-0.453437,-0.394365,-0.076842,-0.775756,0.335051,-1.004812,1.419625,-0.355317,-0.066808,-1.004024
3,TCGA-OY-A56Q,TCGA-OY-A56Q-01Z-00-DX1.F1556F26-8845-4962-900...,DX1,negative,0.009659,1.198278,-0.371777,-0.443551,-0.059649,-1.015064,...,-0.453437,-0.394365,-0.076842,-0.775756,0.830947,-1.004812,-0.472679,-0.355317,-0.066808,-0.759687
4,TCGA-25-1323,TCGA-25-1323-01Z-00-DX1.1B82D1C9-2310-4AAB-B53...,DX1,negative,0.718912,0.733258,2.624842,-0.521355,0.883983,1.007670,...,2.287117,-0.394365,-0.076842,-0.775756,1.106816,1.128314,1.869308,-0.355317,-0.066808,0.170505


In [10]:
df_HGSOC_primary = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_primary_protein_and_phospho.csv")
df_HGSOC_primary.head()

,A1BG,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,AARS1,...,ZNRF2_sp|Q8NHG8|ZNRF2_HUMAN_18_34_1_0,ZRANB2_sp|O95218|ZRAB2_HUMAN_153_165_1_1_S153,ZRANB2_sp|O95218|ZRAB2_HUMAN_183_193_1_1_S188,ZYX_sp|Q15942|ZYX_HUMAN_142_159_1_1_S143,ZYX_sp|Q15942|ZYX_HUMAN_258_259_1_1_S259,ZYX_sp|Q15942|ZYX_HUMAN_267_270_1_1_S267,ZYX_sp|Q15942|ZYX_HUMAN_274_290_1_0,ZYX_sp|Q15942|ZYX_HUMAN_303_316_1_0,ZYX_sp|Q15942|ZYX_HUMAN_344_352_1_1_S344,ZZEF1_sp|O43149|ZZEF1_HUMAN_1499_1523_1_0
0,-0.636902,-1.654378,-0.485982,0.938202,0.399242,0.645449,1.666918,1.905756,1.145809,0.373713,...,-0.102840,-1.130152,0.224968,-0.027355,0.193065,-0.346956,0.074560,2.426106,-0.233432,0.446510
1,-0.636902,-1.654378,-0.485982,0.938202,0.399242,0.645449,1.666918,1.905756,1.145809,0.373713,...,-0.102840,-1.130152,0.224968,-0.027355,0.193065,-0.346956,0.074560,2.426106,-0.233432,0.446510
2,-0.421288,0.293865,1.123371,0.403712,-1.390420,-0.838881,-0.641939,-0.238857,-0.304205,0.670201,...,0.269819,0.281902,-0.873772,-0.397639,-0.030613,0.151614,-0.054610,-0.649941,0.143763,-0.246578
3,-0.421288,0.293865,1.123371,0.403712,-1.390420,-0.838881,-0.641939,-0.238857,-0.304205,0.670201,...,0.269819,0.281902,-0.873772,-0.397639,-0.030613,0.151614,-0.054610,-0.649941,0.143763,-0.246578
4,2.565539,1.980061,-0.771161,-1.703999,-0.097784,-0.179203,1.198760,1.262877,0.554522,-0.579079,...,1.581916,0.051153,0.800913,-0.165402,-1.878681,-0.338320,-0.900831,1.709423,-0.441224,1.662192


In [11]:
# create  csv and split. 
# Load your main CSV file (replace with the actual file path)

# Focus on the 'Sample source' and 'slide_id' columns
cv_data = ffpe_BRCA['slide_id']

# Create a KFold object for 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Directory to store split files
split_dir = 'split_files'
os.makedirs(split_dir, exist_ok=True)

# Iterate over each sample source to create split files
# Filter data for cross-validation and test sets.
test_data = df_HGSOC_primary['slide_id']
# Apply 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(cv_data)):
    # Extract train, val, and test slide IDs
    train_ids = cv_data.iloc[train_index].tolist()
    val_ids = cv_data.iloc[val_index].tolist()
    test_ids = test_data.tolist()

    # Determine the maximum length among train, val, and test lists
    max_len = max(len(train_ids), len(val_ids), len(test_ids))

    # Extend lists to have the same length
    train_ids.extend([""] * (max_len - len(train_ids)))
    val_ids.extend([""] * (max_len - len(val_ids)))
    test_ids.extend([""] * (max_len - len(test_ids)))

    # Create the split DataFrame
    split_df = pd.DataFrame({'train': train_ids[:max_len], 'val': val_ids[:max_len], 'test': test_ids[:max_len]})

    # Save the split file
    split_filename = f'/mnt/ncshare/ozkilim/BRCA/splits/TCGA_BRCA_NERO_train_HGSOC_test_primary/splits_{fold}.csv'
    # split_filepath = os.path.join(split_dir, split_filename)
    split_df.to_csv(split_filename, index=True)

print("Split files created successfully.")

Split files created successfully.


In [12]:
# just slide id and label... 
ffpe_BRCA['brca_status'] = ffpe_BRCA['brca_status'].replace({'positive': 1, 'negative': 0})

ffpe_BRCA = ffpe_BRCA.rename(columns={"brca_status":'label'})



ffpe_nero = ffpe_BRCA[['slide_id','label']]

hgsoc_primary = df_HGSOC_primary[['slide_id','label']]

nero_hgsoc_test = pd.concat([ffpe_nero,hgsoc_primary])
nero_hgsoc_test['case_id'] = nero_hgsoc_test.index
nero_hgsoc_test.head()

,slide_id,label,case_id
0,TCGA-57-1994-01Z-00-DX1.A0798185-30C5-4C32-B1A...,0,0
1,TCGA-23-1027-01Z-00-DX1.53F9DFF4-6811-4184-B2F...,1,1
2,TCGA-42-2593-01Z-00-DX1.9d506f1d-a5b3-4643-bd1...,0,2
3,TCGA-OY-A56Q-01Z-00-DX1.F1556F26-8845-4962-900...,0,3
4,TCGA-25-1323-01Z-00-DX1.1B82D1C9-2310-4AAB-B53...,0,4


In [13]:
nero_hgsoc_test['label'].value_counts()

1    152
0     95
Name: label, dtype: int64

In [14]:
nero_hgsoc_test.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/nero_brca_hgsoc_test.csv",index=None)